# Running kMC Simulation

All routines and kMC code are in the kmcres.py ffile. This notebook is used to try and test runs. 

In [10]:
from kmcres import *
nb.set_num_threads(1)
print("Threading layer -- ",nb.threading_layer())
import os, glob
import sys

Threading layer --  tbb


In [11]:
name = 'exc'
size = 30 #Linear system size, in unis of ld
J = 5 #Inverse temperature times Jsigma
kappa = 0.5 #interaction strength
ld = 4/3 #lattice spacing

Rexc = 3**0.5/2.0
nu = 0.36
udd = 0.116
ec = 2*2**0.5*(udd*(1+udd))/(1+2*udd)

#For now, I put in 2*udd, and it's approximately two ec!
ef = 2*ec #final eigenstrain is always twice the transition-state eigenstrain 
Rf = Rexc #Radius of final excitation

factor = (2**0.5)*(Rf**2)*ef/(1+nu)
dispfactor = (2*Rf)**2/(4*2**0.5*(1+nu))*ef
kappa = kappa*(2*Rf)**2

th = 0.01

print("---- L =",size," ----")
print("---- ld =",ld," ----")
print("---- J =",J," ----")
print("---- kappa =",kappa," ----")

---- L = 30  ----
---- ld = 1.3333333333333333  ----
---- J = 5  ----
---- kappa = 1.4999999999999998  ----


In [13]:
finalT = 1
fh = open('progress.txt', 'w')

frames = 500

#Run simulation with the ability to restart as well
if os.path.exists('finished_{}.txt'.format(name)):
    print("Simulation already finished")
else:
    kmc = KMCModel(J,nu,ld,Rf,kappa,L=size*ld,thetafactor=factor,dispfactor=dispfactor)
    period = int(kmc.Nmax*finalT/frames)
    if period < 1:
        period = 1
    
    print('{}.gsd'.format(name))
    if os.path.exists('{}_restart.gsd'.format(name)):
        kmc.load_logfile(name,savegsd=True)
        initstep = int(kmc.step)
        print(initstep)
        kmc.run(finalT=finalT,period=period,Cb_break=th,verbose=True,initstep=initstep)
    else:
        kmc.create_logfile(name,save=True,savegsd=True)
        kmc.run(finalT=finalT,period=period,Cb_break=th,verbose=True)
    
    f = open("finished_{}.txt".format(name),"w")
    f.close()
    os.remove('{}_restart.gsd'.format(name))
    del f
    if kmc.Cb < th or kmc.reachth:
        print("Reached threshold!")
    del kmc

Simulation already finished
